<a href="https://colab.research.google.com/github/GustaveRw/NLP-Fellowship/blob/master/Assignment_Jean_Marie_Gustave_MBONYINSHUTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Model Assignment
The aim of this assignment is to make sure you understand the foundations of model training. We have covered traditional ML and also simple NN. You task is 
1. to write code for training traditional ML model that gives the highest accuracy
2. Code for NN model that gives the highest accuracy

## This we will consider
1. The code works
2. The hyperparameter used for fine tuning - epoch only is not enough
3. The highest accuracy
4. Bonus points for being creative with preprocessing, tokenization and creation of vectors

# Submissions
1. Notebook with code
2. 2 models

# Deadline
Monday 28th at 5pm

In [4]:
import pandas as pd

full_dataset = pd.read_csv('/content/50k_imdb_movie_reviews.csv')

full_dataset

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test
...,...,...,...
49995,"Towards the end of the movie, I felt it was to...",0,train
49996,This is the kind of movie that my enemies cont...,0,train
49997,I saw 'Descent' last night at the Stockholm Fi...,0,train
49998,Some films that you pick up for a pound turn o...,0,train


In [5]:
full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')]     

,review,sentiment,set
25000,Bromwell High is a cartoon comedy. It ran at t...,1,train
25001,Homelessness (or Houselessness as George Carli...,1,train
25002,Brilliant over-acting by Lesley Ann Warren. Be...,1,train
25003,This is easily the most underrated film inn th...,1,train
25004,This is not the typical Mel Brooks film. It wa...,1,train
...,...,...,...
37495,"Seeing as the vote average was pretty low, and...",1,train
37496,"The plot had some wretched, unbelievable twist...",1,train
37497,I am amazed at how this movie(and most others ...,1,train
37498,A Christmas Together actually came before my t...,1,train


In [6]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12500
No of negative in train : 12500
No of positive in test : 12500
No of negative in test : 12500


In [7]:
full_dataset['review'].describe()

count                                                 50000
unique                                                49582
top       Loved today's show!!! It was a variety and not...
freq                                                      5
Name: review, dtype: object

In [8]:
full_dataset = full_dataset.drop_duplicates(subset=['review'])

In [9]:
full_dataset['review'].describe()

count                                                 49582
unique                                                49582
top       I went and saw this movie last night after bei...
freq                                                      1
Name: review, dtype: object

In [10]:
full_dataset

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test
...,...,...,...
49995,"Towards the end of the movie, I felt it was to...",0,train
49996,This is the kind of movie that my enemies cont...,0,train
49997,I saw 'Descent' last night at the Stockholm Fi...,0,train
49998,Some films that you pick up for a pound turn o...,0,train


In [11]:
train_dataset = full_dataset[(full_dataset['set'] == 'train')][['review','sentiment']]
test_dataset = full_dataset[(full_dataset['set'] == 'test')][['review','sentiment']]
train_dataset.head()

,review,sentiment
25000,Bromwell High is a cartoon comedy. It ran at t...,1
25001,Homelessness (or Houselessness as George Carli...,1
25002,Brilliant over-acting by Lesley Ann Warren. Be...,1
25003,This is easily the most underrated film inn th...,1
25004,This is not the typical Mel Brooks film. It wa...,1


### Cleaning the dataset using Regex

In [12]:

import re
def preprocessing(texts):
  cleaned_text = [] #Construct an empty list to store the reviews
  for text in texts: #for review in df['reviews']
    text = text.lower() #lower case the text
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+') #compile the urls/links patterns
    html_pattern = re.compile('<.*?>') #compile the html tafs

    text = emoji_pattern.sub(r'', text) #replace emojis
    text = url_pattern.sub(r'', text) #replace
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text) # remove special characters!
    
    cleaned_text.append(text)

  return cleaned_text

In [13]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 10 kB/s 
     |████████████████████████████████| 7.1 MB 32.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.


In [14]:
import torch #used to build the model/neural network

from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator


SEED = 42 #Variable from Pop Culture more info at https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a
#It's just a seed to spawn random numbers

torch.manual_seed(SEED)
 #Using deterministic algorithms
torch.backends.cudnn.deterministic = True # reproducibility of the training outcome
max_document_length = 300 #maximun number of words in each sentence
  
#Text is the review                                                    #Tokenise using spacy
TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',     preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)

#Label is the sentiment
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
      items = []
      for i, r in df.iterrows():
        obj = Example.fromlist(list(r), fields)
        items.append(obj)
      print(items)

      super(DataFrameDataset, self).__init__(
            items, 
            fields
        )

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [15]:
#Pass the valid, test dataset
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() 

[<torchtext.legacy.data.example.Example object at 0x7f6901db5090>, <torchtext.legacy.data.example.Example object at 0x7f690221bfd0>, <torchtext.legacy.data.example.Example object at 0x7f6900aa4b50>, <torchtext.legacy.data.example.Example object at 0x7f69b4311950>, <torchtext.legacy.data.example.Example object at 0x7f6900ab2290>, <torchtext.legacy.data.example.Example object at 0x7f6901dcf990>, <torchtext.legacy.data.example.Example object at 0x7f6900ac0a90>, <torchtext.legacy.data.example.Example object at 0x7f6901dcf450>, <torchtext.legacy.data.example.Example object at 0x7f6900acda50>, <torchtext.legacy.data.example.Example object at 0x7f6900acf790>, <torchtext.legacy.data.example.Example object at 0x7f6900aae850>, <torchtext.legacy.data.example.Example object at 0x7f6900a74950>, <torchtext.legacy.data.example.Example object at 0x7f6900a774d0>, <torchtext.legacy.data.example.Example object at 0x7f6900a7ead0>, <torchtext.legacy.data.example.Example object at 0x7f6900a82850>, <torchtex

In [16]:
torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

[<torchtext.legacy.data.example.Example object at 0x7f68e76b1d10>, <torchtext.legacy.data.example.Example object at 0x7f68e7b26150>, <torchtext.legacy.data.example.Example object at 0x7f68e76b1b90>, <torchtext.legacy.data.example.Example object at 0x7f68e76b1c50>, <torchtext.legacy.data.example.Example object at 0x7f68e76b1d90>, <torchtext.legacy.data.example.Example object at 0x7f6901db5950>, <torchtext.legacy.data.example.Example object at 0x7f68e76c2690>, <torchtext.legacy.data.example.Example object at 0x7f68e76b9350>, <torchtext.legacy.data.example.Example object at 0x7f68e76c26d0>, <torchtext.legacy.data.example.Example object at 0x7f68e76c2710>, <torchtext.legacy.data.example.Example object at 0x7f69b42ed6d0>, <torchtext.legacy.data.example.Example object at 0x7f68e75f1cd0>, <torchtext.legacy.data.example.Example object at 0x7f68e75f1d10>, <torchtext.legacy.data.example.Example object at 0x7f68e76c2750>, <torchtext.legacy.data.example.Example object at 0x7f68e75f1d50>, <torchtex

In [17]:
max_size = 30000 #hyperparameter vocab
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)
print(TEXT.vocab.freqs.most_common(20))

.vector_cache/wiki.simple.vec: 293MB [00:06, 46.5MB/s]                           
100%|██████████| 111051/111051 [00:11<00:00, 9555.29it/s]


[(' ', 785692), ('the', 326108), ('and', 161663), ('a', 160563), ('of', 144477), ('to', 134325), ('is', 109343), ('it', 92798), ('in', 92099), ('i', 81980), ('this', 72819), ('that', 72538), ("'s", 61640), (' br', 52238), ('was', 49935), ('as', 45797), ('for', 43582), ('with', 43503), ('movie', 42699), ('but', 41298)]


In [18]:
BATCH_SIZE =  100

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True)
    #device = device)

In [19]:
import torch.nn as nn
import torch.nn.functional as F


class LR(nn.Module):
    def __init__(self, input_size, in_hidden_size,out_hidden_size2, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, in_hidden_size) # 1st layer
        self.relu = nn.ReLU()
                              #input      #output
        self.fc2 = nn.Linear(in_hidden_size, out_hidden_size2)
                              #input       #output with classes to predict only 2 positive/negative 
        self.fc3 = nn.Linear(out_hidden_size2, num_classes)

    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)    #from input layer to first hidden layers
        x = self.relu(x) # passed in an acivation function
        x = self.relu(self.fc2(x)) #hidden layers
        
        preds = self.fc3(x) # crossentropyloss handles the softmax #output layer
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

In [20]:
class MLP(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size2, hidden_size3, hidden_size4, output_dim, dropout, max_document_length):
        super().__init__()
        # embedding and convolution layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(embed_size*max_document_length, hidden_size2)  # dense layer
        self.fc2 = nn.Linear(hidden_size2, hidden_size3)  # dense layer
        self.fc3 = nn.Linear(hidden_size3, hidden_size4)  # dense layer
        self.fc4 = nn.Linear(hidden_size4, output_dim)  # dense layer

    def forward(self, text):
         # text shape = (batch_size, num_sequences)
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        
        x = embedded.view(embedded.shape[0], -1)  # x = Flatten()(x)
        #embedded = embedded.unsqueeze(1) # fc gets 4 dimension
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        preds = self.fc4(x)
        # preds = F.softmax(preds, 1)
        return preds

### Set Hyperparamters for MLP Model


In [21]:
#This is the fine tunnning area

lr = 1e-3   #Learning rate
batch_size = 100   
dropout_keep_prob = 0.2
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size a.k.a unique words
seed = 42
num_classes = 2

num_epochs = 10   
hidden_size = 256  #nodes in the hidden layer 1
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64

to_train = True

model = MLP(vocab_size, embedding_size, hidden_size1, hidden_size2, hidden_size3,  num_classes, dropout_keep_prob, max_document_length)

In [22]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [23]:
best_valid_loss = float('inf') #loss 
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #optimizer using Adaptive with Numeriques(Adam)

loss_func = nn.CrossEntropyLoss() #Cross Entropy

for epoch in range(num_epochs):
  train_epoch_loss = 0 #start from 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model(text).squeeze(1) #feedfoward the input to predict
      
      
      loss = loss_func(predictions, batch.sentiment) #compute the difference from the actual/true output

      acc = accuracy(predictions, batch.sentiment) #current accuracy

      # perform backpropagation
      loss.backward()

      optimizer.step()  #Make an Increment to search for the lowest point on the graph

      train_epoch_loss += loss.item() 
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval() #evaluate the model 

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 0.729 | Train Acc: 50.80%
	 Val. Loss: 0.690 |  Val. Acc: 52.49%
	Train Loss: 0.634 | Train Acc: 60.86%
	 Val. Loss: 0.678 |  Val. Acc: 57.77%
	Train Loss: 0.506 | Train Acc: 70.41%
	 Val. Loss: 0.789 |  Val. Acc: 58.40%
	Train Loss: 0.455 | Train Acc: 74.30%
	 Val. Loss: 0.952 |  Val. Acc: 57.70%
	Train Loss: 0.400 | Train Acc: 78.91%
	 Val. Loss: 0.829 |  Val. Acc: 70.06%
	Train Loss: 0.398 | Train Acc: 79.47%
	 Val. Loss: 0.873 |  Val. Acc: 59.82%
	Train Loss: 0.266 | Train Acc: 87.81%
	 Val. Loss: 0.783 |  Val. Acc: 67.96%
	Train Loss: 0.182 | Train Acc: 92.37%
	 Val. Loss: 0.975 |  Val. Acc: 75.10%
	Train Loss: 0.117 | Train Acc: 95.26%
	 Val. Loss: 0.825 |  Val. Acc: 76.36%
	Train Loss: 0.069 | Train Acc: 97.40%
	 Val. Loss: 1.043 |  Val. Acc: 75.37%


### Set Hyperparamters for LR Model


In [39]:
#This is the fine tunnning area

re_lr = 1e-3   #Learning rate
re_batch_size = 50   
re_dropout_keep_prob = 0.2
re_embedding_size = 300
re_max_document_length = 300 # each sentence has until 100 words
re_vocab_size = len(TEXT.vocab)
re_dev_size = 0.8 # split percentage to train\validation data
re_max_size = 30000 # maximum vocabulary size a.k.a unique words
re_seed = 42
re_num_classes = 2

re_num_epochs = 10   
re_hidden_size = 256  #nodes in the hidden layer 1
re_hidden_size1 = 300
re_hidden_size2 = 128
re_hidden_size3 = 64

to_train = True

model_re = LR(re_max_document_length, re_hidden_size1, re_hidden_size2, re_num_classes)

In [40]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [41]:
best_valid_loss = float('inf') #loss 
optimizer = torch.optim.Adam(model_re.parameters(), lr=lr) #optimizer using Adaptive with Numeriques(Adam)

loss_func = nn.CrossEntropyLoss() #Cross Entropy

for epoch in range(re_num_epochs):
  train_epoch_loss = 0 #start from 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model_re(text).squeeze(1) #feedfoward the input to predict
      
      
      loss = loss_func(predictions, batch.sentiment) #compute the difference from the actual/true output

      acc = accuracy(predictions, batch.sentiment) #current accuracy

      # perform backpropagation
      loss.backward()

      optimizer.step()  #Make an Increment to search for the lowest point on the graph

      train_epoch_loss += loss.item() 
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model_re.eval() #evaluate the model 

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model_re(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model_re.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 12.756 | Train Acc: 49.85%
	 Val. Loss: 0.781 |  Val. Acc: 49.92%
	Train Loss: 0.715 | Train Acc: 50.40%
	 Val. Loss: 0.733 |  Val. Acc: 50.46%
	Train Loss: 0.695 | Train Acc: 51.37%
	 Val. Loss: 0.730 |  Val. Acc: 50.10%
	Train Loss: 0.688 | Train Acc: 51.81%
	 Val. Loss: 0.726 |  Val. Acc: 50.91%
	Train Loss: 0.682 | Train Acc: 52.66%
	 Val. Loss: 0.733 |  Val. Acc: 50.60%
	Train Loss: 0.678 | Train Acc: 53.12%
	 Val. Loss: 0.740 |  Val. Acc: 50.69%
	Train Loss: 0.673 | Train Acc: 53.72%
	 Val. Loss: 0.741 |  Val. Acc: 50.27%
	Train Loss: 0.668 | Train Acc: 54.74%
	 Val. Loss: 0.742 |  Val. Acc: 50.55%
	Train Loss: 0.662 | Train Acc: 55.28%
	 Val. Loss: 0.764 |  Val. Acc: 50.57%
	Train Loss: 0.662 | Train Acc: 55.78%
	 Val. Loss: 0.756 |  Val. Acc: 50.54%
